In [9]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import json
# import dataset
import credentials
import sqlite3
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pickle
from Naive_bayes_model import *
import xgboost as xgb

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [10]:
f = open('nb2.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [11]:
f1 = open('xgb', 'rb')
XGBClassifier = pickle.load(f1)
f1.close()

In [12]:
conn = sqlite3.connect('tweet2.db')
c = conn.cursor()

In [13]:
def pred(data):
    
    tweetProcessor = PreProcessTweets()
    preprocessedTrainingSet = tweetProcessor.processTweets(data)
    
    word_features = buildVocabulary(preprocessedTrainingSet)
#     print("\n",word_features,"\n")
#     buildVocabulary(preprocessedTrainingSet)
    trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
    
    print(preprocessedTrainingSet[0][0])
    
    NB = NBayesClassifier.classify(extract_features(preprocessedTrainingSet[0][0],word_features))
    
    return NB
    

In [14]:

def attributeselection(tweet,query_word):
    attr= []

    #text = "Alleged East Bay serial arsonist arrested #SanFrancisco - http://t.co/ojuHfkHVb2"
    tweet = tweet.lower() # convert text to lower-case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove username
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    tweet = re.sub(r'\\s+',r' ', tweet, flags=re.I)
    tweet = re.sub("([^\x00-\x7F])+"," ",tweet) #remove all non-english characters
    tweet = re.sub(r"\s+"," ", tweet, flags = re.I) #remove multiple spaces
    text = tweet
    text = nltk.word_tokenize(text)
    poslist = nltk.pos_tag(text)
#     print(poslist)

    verblist = []
    for i in range(len(poslist)):
        if poslist[i][1].startswith('V'):
            verblist.append(poslist[i])
    
    word_count = len(poslist)
#     print(word_count)

    if len(verblist) == 0:
        l = (query_word,'VB')
        verblist.append(l)
    
#     print(verblist,"\n")
    verb_count = len(verblist)
#     print(verb_count)

   # query_word = 'arson'

    words_before = words_after = 0
    for i in range(len(poslist)):
        if poslist[i][0].startswith(query_word):
            words_before = i

    words_after = word_count - words_before - 1

#     print(words_before)
#     print(words_after)
    verbs = []    
    attr.append(word_count)
#     for i in verblist:
#         print(i[0])
#         verbs.append(i[0])
#     attr.append(verbs)
    attr.append(verb_count)
    attr.append(words_before + 1)
#     attr_query_word.append(query_word)
    attr.append(words_before)
    attr.append(words_after)
    
    
        
    column_names = ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
    print(attr,column_names)
    df = pd.DataFrame([attr], columns=column_names)
    arr_x=df.to_numpy(dtype=object)
    print(arr_x,arr_x.shape)
    y_pred = XGBClassifier.predict(arr_x)
    print("PRIORITY: ",y_pred)
    
    

In [15]:
class TwitterStreamer():
    
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
        
        stream = Stream(auth, listener,tweet_mode='extended')
        
        stream.filter(languages=["en"],track=['coronavirus'])
        
 
class StdOutListener(StreamListener):
    
    def on_status(self, status):
        print(status.text)

    def on_error(self, status):
        print(status)

    def on_data(self, data):
        all_data             = json.loads(data)
        created_at           = all_data['created_at']
        favorite_count       = all_data['favorite_count']
        favorited            = all_data['favorited']
        filter_level         = all_data['filter_level']
        lang                 = all_data['lang']
        retweet_count        = all_data['retweet_count']
        retweeted            = all_data['retweeted']
        source               = all_data['source']
        text                 = all_data['text']
        truncated            = all_data['truncated']
        user_created_at      = all_data['user']['created_at']
        user_followers_count = all_data['user']['followers_count']
        user_location        = all_data['user']['location']
        user_lang            = all_data['user']['lang']
        user_name            = all_data['user']['name']
        user_screen_name     = all_data['user']['screen_name']
        user_time_zone       = all_data['user']['time_zone']
        user_utc_offset      = all_data['user']['utc_offset']
        user_friends_count   = all_data['user']['friends_count']
        geo_enabled          = all_data['user']['geo_enabled']  
        coordinates          = all_data['coordinates']
        geo                  = all_data['geo']
        place                = all_data['place']
        if place is not None:
            place_name       = all_data['place']['name']
        else:
            place_name       = all_data['place']

        print("Tweet: "+text+"\n")
        
        data = [{'text': text,'Classfication': ''}]
        
        info = pred(data)
        
        if info == 'Relevant' or info == 1 or info == '1':
            attributeselection(text,'coronavirus')
            
        print("\n Class: "+info+"\n")
                         
        c.execute('''INSERT INTO tweets 
        (created_at, favorite_count, favorited, filter_level, lang, 
                         retweet_count, retweeted, source, text, truncated, user_created_at,  
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ) 
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
            (created_at, favorite_count, favorited, filter_level, lang, retweet_count, 
                         retweeted, source, text, truncated, user_created_at, 
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ))
        
        c.execute('''INSERT INTO tweet_class 
        (text, class) 
            VALUES (?,?)''', 
            (text, info))
        
        conn.commit()

In [16]:
if __name__ == '__main__':
    
    hash_tag_list = ['coronavirus']
    fetched_tweets_filename = "tweets.json"
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

Tweet: RT @CREWcrew: We need to talk about Jared Kushner's coronavirus task force, and how it appears to be breaking not one, but two laws by oper…

['rt', 'need', 'talk', 'jared', 'kushner', "'s", 'coronavirus', 'task', 'force', 'appears', 'breaking', 'one', 'two', 'laws', 'oper…']

 Class: Not Relevant

Tweet: RT @BadFoxGraphics: "A team of Oscar engineers, project managers, and executives spent about five days building a stand-alone website at th…

['rt', '``', 'team', 'oscar', 'engineers', 'project', 'managers', 'executives', 'spent', 'five', 'days', 'building', 'stand-alone', 'website', 'th…']
[24, 3, 1, 0, 23] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[24 3 1 0 23]] (1, 5)
PRIORITY:  [0]

 Class: Relevant

Tweet: RT @AaronFullerton: Your coronavirus/April Fool's plan is a bad idea. Trust me.

['rt', 'coronavirus/april', 'fool', "'s", 'plan', 'bad', 'idea', 'trust']

 Class: Not Relevant



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: RT @Nigensei: Empty hotels all over the city of Las Vegas and they’re putting the homeless in a fucking parking lot.

['rt', 'empty', 'hotels', 'city', 'las', 'vegas', '’', 'putting', 'homeless', 'fucking', 'parking', 'lot']

 Class: Not Relevant

Tweet: RT @paperdragun: she prolly don’t even know about the coronavirus https://t.co/E7PU5BY8yZ

['rt', 'prolly', '’', 'even', 'know', 'coronavirus']

 Class: Not Relevant

Tweet: RT @carolecadwalla: ‘Now that we've missed the opportunity for early testing, is it too late for testing to make a difference?’

‘Absolutel…

['rt', '‘', "'ve", 'missed', 'opportunity', 'early', 'testing', 'late', 'testing', 'make', 'difference', '’', '‘', 'absolutel…']
[25, 5, 1, 0, 24] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[25 5 1 0 24]] (1, 5)
PRIORITY:  [0]

 Class: Relevant



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: Go easy on yourself. As a long-time #RemoteWorker, my focus isn't where it was before #Coronavirus. You're not the… https://t.co/pZdXZAM7XV

['go', 'easy', 'long-time', 'remoteworker', 'focus', "n't", 'coronavirus', "'re", 'the…']

 Class: Not Relevant

Tweet: RT @charliekirk11: Why is it that—because of Coronavirus—pastors are being arrested for holding church services but child rapists are being…

['rt', 'that—because', 'coronavirus—pastors', 'arrested', 'holding', 'church', 'services', 'child', 'rapists', 'being…']
[22, 7, 9, 8, 13] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[22 7 9 8 13]] (1, 5)
PRIORITY:  [0]

 Class: Relevant



...

Tweet: RT @zellieimani: Once the government called this the “War on coronavirus”, I just knew it wasn’t going to end anytime soon.

['rt', 'government', 'called', '“', 'war', 'coronavirus', '”', 'knew', '’', 'going', 'end', 'anytime', 'soon']
[24, 5, 11, 10, 13] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[24 5 11 10 13]] (1, 5)
PRIORITY:  [0]

 Class: Relevant

Tweet: RT @anadoluagency: #BREAKING Death toll in Turkey from coronavirus surges to 214 after 46 more people die on Tuesday

['rt', 'breaking', 'death', 'toll', 'turkey', 'coronavirus', 'surges', '214', '46', 'people', 'die', 'tuesday']
[19, 2, 9, 8, 10] ['no_of_words', 'no_of_verbs', 'pos_query_word', 'word_before', 'word_after']
[[19 2 9 8 10]] (1, 5)
PRIORITY:  [0]

 Class: Relevant



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Tweet: RT @StevenBeschloss: Every time I see Trump now, I think of the coronavirus patients, in pain, gasping for air. I think how many will lose…

['rt', 'every', 'time', 'see', 'trump', 'think', 'coronavirus', 'patients', 'pain', 'gasping', 'air', 'think', 'many', 'lose…']

 Class: Not Relevant

Tweet: great little article, thanks

['great', 'little', 'article', 'thanks']

 Class: Not Relevant

Tweet: RT @cllrRLangton: Self-isolating and in need of food? You can now purchase a food parcel from Nottingham City Council, to be delivered to y…

['rt', 'self-isolating', 'need', 'food', 'purchase', 'food', 'parcel', 'nottingham', 'city', 'council', 'delivered', 'y…']

 Class: Not Relevant

Tweet: RT @RealKyleMorris: Twitter Forces @IngrahamAngle to Delete Post on Chloroquine Helping Coronavirus Patients https://t.co/ztpX0LIPIV

['rt', 'twitter', 'forces', 'delete', 'post', 'chloroquine', 'helping', 'coronavirus', 'patients']
[14, 3, 12, 11, 2] ['no_of_words', 'no_of_verbs', 'pos_query_word

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


KeyboardInterrupt: 